## Задание 1

Открой файл [distances.ipynb](src/notebooks/distances.ipynb). 
* Объедини общие данные о фильмах [tmdb_5000_movies](https://files.sberdisk.ru/s/te4QbzdxKgsFQXA) и каст фильмов 
[tmdb_5000_credits](https://files.sberdisk.ru/s/H9oRuXQt5mFz3T9). 
* Оставь в датасете только фильмы, которые вышли в "релиз".
* Убери фильмы с пропусками в колонках ['overview', 'genres', 'keywords']
* Выведи количество фильмов, оставшихся в выборке 

In [10]:
import pandas as pd

In [11]:
# Создаем датафремы из csv файлов
df_movies = pd.read_csv('../../datasets/tmdb_5000_movies.csv')
df_credits = pd.read_csv('../../datasets/tmdb_5000_credits.csv')

# Объединяем датафреймы в один, причем movies слева и объединение по столбцу id, credits справа и объединение по столбцу movie_id
df = pd.merge(df_movies, df_credits, how='left', left_on='id', right_on='movie_id')

# Оставим в датафреймы только те фильмы, которые вышли в релиз
df = df[df['status'] == 'Released']

# Убираем фильмы с пропусками в колонках ['overview', 'genres', 'keywords']
df = df.dropna(subset =['overview'])
df = df.dropna(subset =['genres'])
df = df.dropna(subset =['keywords'])

print('Количество фильмов, оставшихся в выборке =', df.shape[0])




Количество фильмов, оставшихся в выборке = 4792


## Задание 2

Реализуем алгоритм рекомендации на основе описания фильма (`overview`) и ключевых слов к фильму (`keywords`). 
Объедини тексты этих колонок и проведи предобработку:
* Замени NaN в описании фильма на пустой символ `''`
* Удали все английские стоп-слова (используй параметр `stop_words` в `TfidfVectorizer`)
* Рассчитай матрицу [Tf-Idf](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) для описания фильмов.

Выведи размер получившейся матрицы
> Параметр `max_features` в `TfidfVectorizer` должен быть равен 10000

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [13]:
# объединим тексты колонок в одну
df['overview'].head(5)

0    In the 22nd century, a paraplegic Marine is di...
1    Captain Barbossa, long believed to be dead, ha...
2    A cryptic message from Bond’s past sends him o...
3    Following the death of District Attorney Harve...
4    John Carter is a war-weary, former military ca...
Name: overview, dtype: object

In [14]:
df['keywords'].head(5)

0    [{"id": 1463, "name": "culture clash"}, {"id":...
1    [{"id": 270, "name": "ocean"}, {"id": 726, "na...
2    [{"id": 470, "name": "spy"}, {"id": 818, "name...
3    [{"id": 849, "name": "dc comics"}, {"id": 853,...
4    [{"id": 818, "name": "based on novel"}, {"id":...
Name: keywords, dtype: object

In [15]:
df['overview_and_keywords'] = df['overview'] + df['keywords']
df['overview_and_keywords']

0       In the 22nd century, a paraplegic Marine is di...
1       Captain Barbossa, long believed to be dead, ha...
2       A cryptic message from Bond’s past sends him o...
3       Following the death of District Attorney Harve...
4       John Carter is a war-weary, former military ca...
                              ...                        
4798    El Mariachi just wants to play his guitar and ...
4799    A newlywed couple's honeymoon is upended by th...
4800    "Signed, Sealed, Delivered" introduces a dedic...
4801    When ambitious New York attorney Sam is sent t...
4802    Ever since the second grade when he first saw ...
Name: overview_and_keywords, Length: 4792, dtype: object

In [16]:
vector = TfidfVectorizer(stop_words='english', max_features= 10000)  # создаем матрицу функций, удаляя все английские слова
df['overview'] = df['overview'].fillna('')  # заменяем NaN в описании фильма на пустой символ ""
matrix = vector.fit_transform(df.overview)  # создаем матрицу TF-Idf
print ('Размер получившейся матрицы Tf-Idf', matrix.shape)

Размер получившейся матрицы Tf-Idf (4792, 10000)


## Задание 3

Рассчитай [cosine similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html) 
между фильмами. Составь из этой матрицы `pd.DataFrame`. Для дальнейшего удобства, 
колонки и индексы таблицы назови согласно`id` фильма. \
Сохрани получившийся `DataFrame` c расстояниями в папку [assets](src/assets) с названием `distance.csv`.
А сам объединенный датасет с фильмами сохрани в папку [assets](src/assets) с названием `movies.csv`.

> Получившиеся файлы `distance.csv` и `movies.csv` пушить в GitLab не нужно!

In [17]:
# Код тут
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(matrix)
# df_distance = pd.DataFrame(cosine_sim, index = df.id, columns = df.id)
df_distance = pd.DataFrame(cosine_sim, index = df['id'], columns = df['id'])

In [18]:
df_distance.to_csv('../assets/distance.csv')
df.to_csv('../assets/movies.csv')